In [49]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [50]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [51]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/6章'
# os.chdir(path)

# ６章 機械学習のためのデータ加工をする１０本ノック



### ノック５１：データ加工の下準備をしよう

In [52]:
import os
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "1_output")
master_dir = os.path.join(data_dir, "99_master")
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

##### ＊＊必ずデータの配置をおこなってください。＊＊

#### MacやGoogle Colaboratory等、環境によっては、ファイル取得時の順番が年月順にならないことがあります。
#### その場合、tbl_order_paths = sorted(tbl_order_paths)を入れてください。

In [53]:
import glob
tbl_order_file = os.path.join(input_dir, 'tbl_order_*.csv')
tbl_order_paths = glob.glob(tbl_order_file)
tbl_order_paths

['data/0_input/tbl_order_201911.csv',
 'data/0_input/tbl_order_201905.csv',
 'data/0_input/tbl_order_202001.csv',
 'data/0_input/tbl_order_201904.csv',
 'data/0_input/tbl_order_201910.csv',
 'data/0_input/tbl_order_201906.csv',
 'data/0_input/tbl_order_201912.csv',
 'data/0_input/tbl_order_202002.csv',
 'data/0_input/tbl_order_202003.csv',
 'data/0_input/tbl_order_201907.csv',
 'data/0_input/tbl_order_201909.csv',
 'data/0_input/tbl_order_201908.csv']

### ノック５２：データの読み込みを行い加工の方向性を検討しよう

In [54]:
import pandas as pd
m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [55]:
tbl_order_path = tbl_order_paths[0]
print(f'読み込みデータ：{tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
print(f'データ件数：{len(order_data)}')
order_data.head(3)

読み込みデータ：data/0_input/tbl_order_201911.csv
データ件数：233191


,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status
0,40087130,127,C80655945,59,27127123,2019-11-01 11:00:00,2019-11-01 11:37:00,1,2154,9
1,51902951,107,C90047893,75,4783489,2019-11-01 11:00:00,2019-11-01 11:28:00,0,2877,2
2,75684882,163,C77025391,52,20052579,2019-11-01 11:00:00,2019-11-01 11:43:00,0,2154,9


### ノック５３：１か月分のデータの基本的なデータ加工を実施しよう

In [56]:
order_data = order_data.loc[order_data['store_id'] != 999]

order_data = pd.merge(order_data, m_store, on='store_id', how='left')
order_data = pd.merge(order_data, m_area, on='area_cd', how='left')

order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'

order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name
0,40087130,127,C80655945,59,27127123,2019-11-01 11:00:00,2019-11-01 11:37:00,1,2154,9,戸塚店,KN,神奈川,神奈川,お持ち帰り,キャンセル
1,51902951,107,C90047893,75,4783489,2019-11-01 11:00:00,2019-11-01 11:28:00,0,2877,2,中店,KN,神奈川,神奈川,デリバリー,お渡し済
2,75684882,163,C77025391,52,20052579,2019-11-01 11:00:00,2019-11-01 11:43:00,0,2154,9,浦和店,SA,埼玉,埼玉,デリバリー,キャンセル


In [57]:
order_data.isna().sum()

order_id             0
store_id             0
customer_id          0
coupon_cd            0
sales_detail_id      0
order_accept_date    0
delivered_date       0
takeout_flag         0
total_amount         0
status               0
store_name           0
area_cd              0
wide_area            0
narrow_area          0
takeout_name         0
status_name          0
dtype: int64

### ノック５４：機械学習に使用する変数を作成しよう

In [58]:
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds()/60

order_data.loc[:,'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
order_data.loc[:,'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
order_data.loc[:,'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,store_name,area_cd,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta
0,40087130,127,C80655945,59,27127123,2019-11-01 11:00:00,2019-11-01 11:37:00,1,2154,9,戸塚店,KN,神奈川,神奈川,お持ち帰り,キャンセル,2019-11-01 11:00:00,2019-11-01 11:37:00,37.0
1,51902951,107,C90047893,75,4783489,2019-11-01 11:00:00,2019-11-01 11:28:00,0,2877,2,中店,KN,神奈川,神奈川,デリバリー,お渡し済,2019-11-01 11:00:00,2019-11-01 11:28:00,28.0
2,75684882,163,C77025391,52,20052579,2019-11-01 11:00:00,2019-11-01 11:43:00,0,2154,9,浦和店,SA,埼玉,埼玉,デリバリー,キャンセル,2019-11-01 11:00:00,2019-11-01 11:43:00,43.0


In [59]:
order_data.loc[:,'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
order_data.loc[:,'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '休日'
order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '平日'
order_data.head(3)

,order_id,store_id,customer_id,coupon_cd,sales_detail_id,order_accept_date,delivered_date,takeout_flag,total_amount,status,...,wide_area,narrow_area,takeout_name,status_name,order_accept_datetime,delivered_datetime,delta,order_accept_hour,order_accept_weekday,weekday_info
0,40087130,127,C80655945,59,27127123,2019-11-01 11:00:00,2019-11-01 11:37:00,1,2154,9,...,神奈川,神奈川,お持ち帰り,キャンセル,2019-11-01 11:00:00,2019-11-01 11:37:00,37.0,11,4,平日
1,51902951,107,C90047893,75,4783489,2019-11-01 11:00:00,2019-11-01 11:28:00,0,2877,2,...,神奈川,神奈川,デリバリー,お渡し済,2019-11-01 11:00:00,2019-11-01 11:28:00,28.0,11,4,平日
2,75684882,163,C77025391,52,20052579,2019-11-01 11:00:00,2019-11-01 11:43:00,0,2154,9,...,埼玉,埼玉,デリバリー,キャンセル,2019-11-01 11:00:00,2019-11-01 11:43:00,43.0,11,4,平日


### ノック５５：店舗単位に集計して変数を作成しよう

In [60]:
store_data = order_data.groupby(['store_name']).count()[['order_id']]
store_f = order_data.loc[(order_data['status_name']=="お渡し済")|
                         (order_data['status_name']=="お支払済")].groupby(['store_name']).count()[['order_id']]
store_c = order_data.loc[order_data['status_name']=="キャンセル"].groupby(['store_name']).count()[['order_id']]
store_d = order_data.loc[order_data['takeout_name']=="デリバリー"].groupby(['store_name']).count()[['order_id']]
store_t = order_data.loc[order_data['takeout_name']=="お持ち帰り"].groupby(['store_name']).count()[['order_id']]

store_weekday = order_data.loc[order_data['weekday_info']=="平日"].groupby(['store_name']).count()[['order_id']]
store_weekend = order_data.loc[order_data['weekday_info']=="休日"].groupby(['store_name']).count()[['order_id']]

In [61]:
times = order_data['order_accept_hour'].unique()
store_time = []
for time in times:
    time_tmp = order_data.loc[order_data['order_accept_hour']==time].groupby(['store_name']).count()[['order_id']]
    time_tmp.columns = [f'order_time_{time}']
    store_time.append(time_tmp)
store_time = pd.concat(store_time, axis=1)
store_time.head(3)

,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21
store_name,,,,,,,,,,,
あきる野店,84,105,102,93,107,92,104,117,111,118,104
さいたま南店,150,139,139,132,127,117,130,132,135,161,146
さいたま緑店,74,75,114,98,94,101,93,92,109,85,92


In [62]:
store_delta = order_data.loc[(order_data['status_name']!="キャンセル")].groupby(['store_name']).mean()[['delta']]
store_data.columns = ['order']
store_f.columns = ['order_fin']
store_c.columns = ['order_cancel']
store_d.columns = ['order_delivery']
store_t.columns = ['order_takeout']
store_weekday.columns = ['order_weekday']
store_weekend.columns = ['order_weekend']
store_delta.columns = ['delta_avg']
store_data = pd.concat([store_data, store_f, store_c, store_d, store_t, 
                        store_weekday, store_weekend, store_time, store_delta], axis=1)
store_data.head(3)

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg
store_name,,,,,,,,,,,,,,,,,,,
あきる野店,1137,943,194,853,284,796,341,84,105,102,93,107,92,104,117,111,118,104,34.513256
さいたま南店,1508,1243,265,1110,398,1055,453,150,139,139,132,127,117,130,132,135,161,146,34.276750
さいたま緑店,1027,837,190,740,287,719,308,74,75,114,98,94,101,93,92,109,85,92,34.511350


### ノック５６：データの加工と店舗別集計を関数で実行しよう

In [63]:
def data_processing(order_data):
    order_data = order_data.loc[order_data['store_id'] != 999]
    order_data = pd.merge(order_data, m_store, on='store_id', how='left')
    order_data = pd.merge(order_data, m_area, on='area_cd', how='left')
    order_data.loc[order_data['takeout_flag'] == 0, 'takeout_name'] = 'デリバリー'
    order_data.loc[order_data['takeout_flag'] == 1, 'takeout_name'] = 'お持ち帰り'
    order_data.loc[order_data['status'] == 0, 'status_name'] = '受付'
    order_data.loc[order_data['status'] == 1, 'status_name'] = 'お支払済'
    order_data.loc[order_data['status'] == 2, 'status_name'] = 'お渡し済'
    order_data.loc[order_data['status'] == 9, 'status_name'] = 'キャンセル'
    
    order_data.loc[:,'order_accept_datetime'] = pd.to_datetime(order_data['order_accept_date'])
    order_data.loc[:,'delivered_datetime'] = pd.to_datetime(order_data['delivered_date'])
    order_data.loc[:,'delta'] = order_data[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)
    order_data.loc[:,'order_accept_hour'] = order_data['order_accept_datetime'].dt.hour
    order_data.loc[:,'order_accept_weekday'] = order_data['order_accept_datetime'].dt.weekday
    order_data.loc[order_data['order_accept_weekday'] >= 5, 'weekday_info'] = '休日'
    order_data.loc[order_data['order_accept_weekday'] < 5, 'weekday_info'] = '平日'
    
    store_data = order_data.groupby(['store_name']).count()[['order_id']]
    store_f = order_data.loc[(order_data['status_name']=="お渡し済")|
                             (order_data['status_name']=="お支払済")].groupby(['store_name']).count()[['order_id']]
    store_c = order_data.loc[order_data['status_name']=="キャンセル"].groupby(['store_name']).count()[['order_id']]
    store_d = order_data.loc[order_data['takeout_name']=="デリバリー"].groupby(['store_name']).count()[['order_id']]
    store_t = order_data.loc[order_data['takeout_name']=="お持ち帰り"].groupby(['store_name']).count()[['order_id']]
    store_weekday = order_data.loc[order_data['weekday_info']=="平日"].groupby(['store_name']).count()[['order_id']]
    store_weekend = order_data.loc[order_data['weekday_info']=="休日"].groupby(['store_name']).count()[['order_id']]
    times = order_data['order_accept_hour'].unique()
    store_time = []
    for time in times:
        time_tmp = order_data.loc[order_data['order_accept_hour']==time].groupby(['store_name']).count()[['order_id']]
        time_tmp.columns = [f'order_time_{time}']
        store_time.append(time_tmp)
    store_time = pd.concat(store_time, axis=1)
    store_delta = order_data.loc[order_data['status_name']!="キャンセル"].groupby(['store_name']).mean()[['delta']]
    store_data.columns = ['order']
    store_f.columns = ['order_fin']
    store_c.columns = ['order_cancel']
    store_d.columns = ['order_delivery']
    store_t.columns = ['order_takeout']
    store_delta.columns = ['delta_avg']
    store_weekday.columns = ['order_weekday']
    store_weekend.columns = ['order_weekend']
    store_data = pd.concat([store_data, store_f, store_c, store_d, store_t, 
                        store_weekday, store_weekend, store_time, store_delta], axis=1)
    return store_data

In [64]:
tbl_order_path = tbl_order_paths[0]
print(f'読み込みデータ：{tbl_order_path}')
order_data = pd.read_csv(tbl_order_path)
store_data = data_processing(order_data)
store_data.head(3)

読み込みデータ：data/0_input/tbl_order_201911.csv


,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg
store_name,,,,,,,,,,,,,,,,,,,
あきる野店,1137,943,194,853,284,796,341,84,105,102,93,107,92,104,117,111,118,104,34.513256
さいたま南店,1508,1243,265,1110,398,1055,453,150,139,139,132,127,117,130,132,135,161,146,34.276750
さいたま緑店,1027,837,190,740,287,719,308,74,75,114,98,94,101,93,92,109,85,92,34.511350


### ノック５７：全データの読み込みとデータ加工をやってみよう

In [65]:
store_all = []
for tbl_order_path in tbl_order_paths:
    print(f'読み込みデータ：{tbl_order_path}')
    tg_ym = tbl_order_path.split('_')[-1][:6]
    order_data = pd.read_csv(tbl_order_path)
    store_data = data_processing(order_data)
    store_data.loc[:,'year_month'] = tg_ym
    store_data.reset_index(drop=False, inplace=True)
    store_all.append(store_data)
store_all = pd.concat(store_all, ignore_index=True)
display(store_all.head(3))
display(store_all.tail(3))
store_monthly_name = 'store_monthly_data.csv'
store_all.to_csv(os.path.join(output_dir, store_monthly_name), index=False)

読み込みデータ：data/0_input/tbl_order_201911.csv
読み込みデータ：data/0_input/tbl_order_201905.csv
読み込みデータ：data/0_input/tbl_order_202001.csv
読み込みデータ：data/0_input/tbl_order_201904.csv
読み込みデータ：data/0_input/tbl_order_201910.csv
読み込みデータ：data/0_input/tbl_order_201906.csv
読み込みデータ：data/0_input/tbl_order_201912.csv
読み込みデータ：data/0_input/tbl_order_202002.csv
読み込みデータ：data/0_input/tbl_order_202003.csv
読み込みデータ：data/0_input/tbl_order_201907.csv
読み込みデータ：data/0_input/tbl_order_201909.csv
読み込みデータ：data/0_input/tbl_order_201908.csv


,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1137,943,194,853,284,796,341,84,105,...,93,107,92,104,117,111,118,104,34.513256,201911
1,さいたま南店,1508,1243,265,1110,398,1055,453,150,139,...,132,127,117,130,132,135,161,146,34.276750,201911
2,さいたま緑店,1027,837,190,740,287,719,308,74,75,...,98,94,101,93,92,109,85,92,34.511350,201911


,store_name,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
2337,鴻巣店,1159,942,217,845,314,822,337,99,106,...,109,97,111,88,111,111,105,111,34.251592,201908
2338,鶴見店,1516,1263,253,1107,409,1079,437,128,151,...,126,125,138,159,145,140,127,132,34.936659,201908
2339,麻生店,1398,1140,258,1006,392,992,406,119,130,...,119,124,133,122,132,131,132,135,34.049123,201908


### ノック５８：目的変数を作成しよう

In [66]:
y = store_all[['store_name', 'year_month','order_weekday', 'order_weekend']].copy()
y.loc[:,'one_month_ago'] = pd.to_datetime(y['year_month'], format='%Y%m')
from dateutil.relativedelta import relativedelta
y.loc[:,'one_month_ago'] = y['one_month_ago'].map(lambda x: x - relativedelta(months=1))
y.loc[:,'one_month_ago'] = y['one_month_ago'].dt.strftime('%Y%m')
y.head(3)

,store_name,year_month,order_weekday,order_weekend,one_month_ago
0,あきる野店,201911,796,341,201910
1,さいたま南店,201911,1055,453,201910
2,さいたま緑店,201911,719,308,201910


In [67]:
y_one_month_ago = y.copy()
y_one_month_ago.rename(columns={'order_weekday':'order_weekday_one_month_ago', 
                                'order_weekend':'order_weekend_one_month_ago',
                                'year_month':'year_month_for_join'}, inplace=True)
y = pd.merge(y, y_one_month_ago[['store_name', 'year_month_for_join', 
                                 'order_weekday_one_month_ago', 'order_weekend_one_month_ago']], 
                                 left_on=['store_name', 'one_month_ago'], 
                                 right_on=['store_name','year_month_for_join'], how='left')
y.loc[y['store_name']=='あきる野店']

,store_name,year_month,order_weekday,order_weekend,one_month_ago,year_month_for_join,order_weekday_one_month_ago,order_weekend_one_month_ago
0,あきる野店,201911,796,341,201910,201910,880.0,309.0
195,あきる野店,201905,883,302,201904,201904,844.0,303.0
390,あきる野店,202001,881,305,201912,201912,844.0,345.0
585,あきる野店,201904,844,303,201903,NaN,NaN,NaN
780,あきる野店,201910,880,309,201909,201909,802.0,347.0
975,あきる野店,201906,764,384,201905,201905,883.0,302.0
1170,あきる野店,201912,844,345,201911,201911,796.0,341.0
1365,あきる野店,202002,762,344,202001,202001,881.0,305.0
1560,あきる野店,202003,839,347,202002,202002,762.0,344.0
1755,あきる野店,201907,882,308,201906,201906,764.0,384.0


In [68]:
y.dropna(inplace=True)
y.loc[y['order_weekday'] - y['order_weekday_one_month_ago'] > 0, 'y_weekday'] = 1
y.loc[y['order_weekday'] - y['order_weekday_one_month_ago'] <= 0, 'y_weekday'] = 0
y.loc[y['order_weekend'] - y['order_weekend_one_month_ago'] > 0, 'y_weekend'] = 1
y.loc[y['order_weekend'] - y['order_weekend_one_month_ago'] <= 0, 'y_weekend'] = 0
y.head(3)

,store_name,year_month,order_weekday,order_weekend,one_month_ago,year_month_for_join,order_weekday_one_month_ago,order_weekend_one_month_ago,y_weekday,y_weekend
0,あきる野店,201911,796,341,201910,201910,880.0,309.0,0.0,1.0
1,さいたま南店,201911,1055,453,201910,201910,1155.0,401.0,0.0,1.0
2,さいたま緑店,201911,719,308,201910,201910,789.0,276.0,0.0,1.0


### ノック５９：説明変数と目的変数を紐づけて機械学習用のデータを仕上げよう

In [69]:
y.rename(columns={'year_month':'target_year_month'},inplace=True)
y = y[['store_name','target_year_month', 'one_month_ago', 'y_weekday', 'y_weekend']].copy()
ml_data = pd.merge(y, store_all, left_on=['store_name','one_month_ago'],
                   right_on=['store_name','year_month'], how='left')
ml_data.head()

,store_name,target_year_month,one_month_ago,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,201911,201910,0.0,1.0,1189,970,219,848,341,...,108,96,111,90,126,112,116,104,34.569072,201910
1,さいたま南店,201911,201910,0.0,1.0,1556,1278,278,1155,401,...,167,143,127,174,136,141,150,120,34.309077,201910
2,さいたま緑店,201911,201910,0.0,1.0,1065,852,213,789,276,...,100,94,100,92,107,88,94,95,34.546948,201910
3,さいたま西店,201911,201910,0.0,1.0,1212,995,217,901,311,...,105,110,99,88,105,118,128,146,34.597990,201910
4,つくば店,201911,201910,0.0,1.0,1304,1066,238,965,339,...,89,121,103,128,134,107,120,134,34.991557,201910


In [70]:
del ml_data["target_year_month"]
del ml_data["one_month_ago"]
ml_data.head()

,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,0.0,1.0,1189,970,219,848,341,880,309,...,108,96,111,90,126,112,116,104,34.569072,201910
1,さいたま南店,0.0,1.0,1556,1278,278,1155,401,1155,401,...,167,143,127,174,136,141,150,120,34.309077,201910
2,さいたま緑店,0.0,1.0,1065,852,213,789,276,789,276,...,100,94,100,92,107,88,94,95,34.546948,201910
3,さいたま西店,0.0,1.0,1212,995,217,901,311,898,314,...,105,110,99,88,105,118,128,146,34.597990,201910
4,つくば店,0.0,1.0,1304,1066,238,965,339,967,337,...,89,121,103,128,134,107,120,134,34.991557,201910


### ノック６０：機械学習用データの確認を行い出力しよう

In [71]:
ml_data.isna().sum()

store_name        0
y_weekday         0
y_weekend         0
order             0
order_fin         0
order_cancel      0
order_delivery    0
order_takeout     0
order_weekday     0
order_weekend     0
order_time_11     0
order_time_12     0
order_time_13     0
order_time_14     0
order_time_15     0
order_time_16     0
order_time_17     0
order_time_18     0
order_time_19     0
order_time_20     0
order_time_21     0
delta_avg         0
year_month        0
dtype: int64

In [72]:
display(ml_data.groupby("y_weekday").count()[["store_name"]])
display(ml_data.groupby("y_weekend").count()[["store_name"]])


,store_name
y_weekday,
0.0,975
1.0,1170


,store_name
y_weekend,
0.0,1003
1.0,1142


In [73]:
ml_data.to_csv(os.path.join(output_dir, 'ml_base_data.csv'), index=False)